# **Assignment-2 for CS60075: Natural Language Processing**

#### Instructor : Prof. Sudeshna Sarkar

#### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Prithwish Jana, Udit Dharmin Desai

#### Date of Announcement: 15th Sept, 2021
#### Deadline for Submission: 11.59pm on Wednesday, 22nd Sept, 2021 
#### Submit this .ipynb file, named as `<Your_Roll_Number>_Assn2_NLP_A21.ipynb`

The central idea of this assignment is to use Naive Bayes classifier and LSTM based classifier and compare the models by accuracy on IMDB dataset.  This dataset consists of 50k movie reviews (25k positive, 25k negative). You can download the dataset from https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews



Please submit with outputs. 

In [ ]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report

In [ ]:
#Load the IMDB dataset. You can load it using pandas as dataframe
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP/Assignement-2/IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Preprocessing
PrePrecessing that needs to be done on lower cased corpus

1. Remove html tags
2. Remove URLS
3. Remove non alphanumeric character
4. Remove Stopwords
5. Perform stemming and lemmatization

You can use regex from re. 

In [ ]:
import nltk

nltk.download('punkt')

nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()

from nltk.stem import PorterStemmer
ps = PorterStemmer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# **Converting the corpus to lower cased**

In [ ]:
def convert_to_lower(text):
  text = text.str.lower()
  return text

df['review'] = convert_to_lower(df['review'])

df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


# **1. Removing the html tags**

In [ ]:
def remove_html(text):
  text = text.str.replace(r'<[^<>]*>', ' ', regex=True)
  text = text.str.replace(r'\s+',' ', regex=True)

  return text

df['review'] = remove_html(df['review'])

df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


# **2. Removing any urls present in the corpus**


In [ ]:
def remove_url(text):
  text = text.str.replace(r'http\S+|\S+\.com|www\.[a-z0-9]+\.\S+',' ', regex=True)
  text = text.str.replace(r'\s+',' ', regex=True)
  return text

df['review'] = remove_url(df['review'])

df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


# **3. Removing non alpha numeric characters**

In [ ]:
def remove_non_alpha(text):
  text = text.str.replace(r'[^a-z0-9 ]',' ', regex=True)
  text = text.str.replace(r'\s+',' ', regex=True)
  return text

df['review'] = remove_non_alpha(df['review'])

df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production the filming tech...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there s a family where a little boy ...,negative
4,petter mattei s love in the time of money is a...,positive


**Removing any redundant white spaces present in the corpus**

In [ ]:
def remove_extra_space(text):
  text = text.str.replace(r'\b[a-z]\b',' ', regex=True)
  text = text.str.replace(r'\n',' ', regex=True)
  text = text.str.replace(r'\n\n',' ', regex=True)
  text = text.str.replace(r'\s+',' ', regex=True)

  return text

df['review'] = remove_extra_space(df['review'])

df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,wonderful little production the filming techn...,positive
2,thought this was wonderful way to spend time ...,positive
3,basically there family where little boy jake t...,negative
4,petter mattei love in the time of money is vis...,positive


# **4. Removing the stopwords**

In [ ]:
def remove_stopword(text):
  text = text.apply(lambda review: ' '.join([word for word in review.split() if not word in stop_words]))
  return text

df['review'] = remove_stopword(df['review'])

df.head()

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production filming technique ...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically family little boy jake thinks zombie...,negative
4,petter mattei love time money visually stunnin...,positive


# **5. Performing stemming and lemmatization**

In [ ]:
def lemmatize_and_stem_text(text):
  text = text.apply(lambda review : ' '.join([wnl.lemmatize(word) for word in review.split()]))

  text = text.apply(lambda review: ' '.join([ps.stem(word) for word in review.split()]))

  return text

df['review'] = lemmatize_and_stem_text(df['review'])

df.head()

,review,sentiment
0,one review mention watch 1 oz episod hook righ...,positive
1,wonder littl product film techniqu unassum old...,positive
2,thought wonder way spend time hot summer weeke...,positive
3,basic famili littl boy jake think zombi closet...,negative
4,petter mattei love time money visual stun film...,positive


# **Statistics of Data**

In [ ]:
# Print Statistics of Data like avg length of sentence , proposition of data w.r.t class labels

print("Average length of sentence: ")

total_len = 0
total_sentences = 0

for review in df['review']:
  total_len += len(review.split())
  total_sentences += 1

print(total_len//total_sentences)


Average length of sentence: 
119


In [ ]:
print("Proposition of data w.r.t class labels: ")
df['sentiment'].value_counts()

Proposition of data w.r.t class labels: 


positive    25000
negative    25000
Name: sentiment, dtype: int64

# Naive Bayes classifier

In [ ]:
# get reviews column from df
reviews = df['review']

# get labels column from df
labels = df['sentiment']

In [ ]:
# Use label encoder to encode labels. Convert to 0/1
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

print(encoder.classes_)

['negative' 'positive']


In [ ]:
# Split the data into train and test (80% - 20%). 
# Use stratify in train_test_split so that both train and test have similar ratio of positive and negative samples.


# Note: 
# train_sentences, test_sentences ---> pandas.series (since, reviews is )
# train_labels, test_labels ---> numpy.ndarray (since, encoded_labels is numpy.ndarray)
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, test_size=0.2, random_state=0, stratify=encoded_labels)



Here there are two approaches possible for building vocabulary for the naive Bayes.
1. Take the whole data (train + test) to build the vocab. In this way while testing there is no word which will be out of vocabulary.
2. Take the train data to build vocab. In this case, some words from the test set may not be in vocab and hence one needs to perform smoothing so that one the probability term is not zero.
 
You are supposed to go by the 2nd approach.
 
Also building vocab by taking all words in the train set is memory intensive, hence you are required to build vocab by choosing the top 2000 - 3000 frequent words in the training corpus.

> $ P(x_i | w_j) = \frac{ N_{x_i,w_j}\, +\, \alpha }{ N_{w_j}\, +\, \alpha*d} $


$N_{x_i,w_j}$ : Number of times feature $x_i$ appears in samples of class $w_j$

$N_{w_j}$ : Total count of features in class $w_j$

$\alpha$ : Parameter for additive smoothing. Here consider $\alpha$ = 1

$d$ : Dimentionality of the feature vector  $x = [x_1,x_2,...,x_d]$. In our case its the vocab size.






In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
# Use Count vectorizer to get frequency of the words

vec = CountVectorizer(max_features = 3000)

X = vec.fit_transform(train_sentences.to_list())

In [ ]:
feature_list = vec.get_feature_names()

# **Building dictionaries to get counts for probability**

In [ ]:
def build_vocabulary(X, feature_list, train_labels):
  
  pos_dict = dict.fromkeys(feature_list, 0)
  neg_dict = dict.fromkeys(feature_list, 0)

  X = X.toarray()

  for row in range(X.shape[0]):

    if train_labels[row] == 1:
      for col in range(X.shape[1]):
        pos_dict[feature_list[col]] += X[row][col]
    
    else:
      for col in range(X.shape[1]):
        neg_dict[feature_list[col]] += X[row][col]   

  return pos_dict, neg_dict

pos_dict, neg_dict = build_vocabulary(X, feature_list, train_labels)

# **Training naive_bayes_model**

In [ ]:
# Use laplace smoothing for words in test set not present in vocab of train set

def naive_bayes_model(sentences, pos_dict, neg_dict):


  """
    Arguments:
      sentences --> list (List of all the reviews whose probabilities are to be predicted by the model)
      pos_dict --> dict (Contains the frequency of top 3000 features corrosponding to the positive reviews)
      neg_dict --> dict (Contains the frequency of top 3000 features corrosponding to the negative reviews)


    Returns:
      list of predicted labels for the given reviews of whether they are positive (1) or negative (0)
  """

  pos_dict_count = sum(pos_dict.values())
  neg_dict_count = sum(neg_dict.values())

  label_vec = []

  for sent in sentences:

    log_pos_prob = np.log(0.5)
    log_neg_prob = np.log(0.5)

    word_tokens = nltk.tokenize.word_tokenize(sent)
    
    for word in word_tokens:

      pos_word_count = 0

      if word in pos_dict.keys():
        pos_word_count = pos_dict[word]


      neg_word_count = 0

      if word in neg_dict.keys():
        neg_word_count = neg_dict[word]

      log_pos_prob = log_pos_prob + np.log( (1 + pos_word_count) / (pos_dict_count + 3000) )
      log_neg_prob = log_neg_prob + np.log( (1 + neg_word_count) / (neg_dict_count + 3000) )

    if log_pos_prob >= log_neg_prob:
      label_vec.append(1)

    else:
      label_vec.append(0)

  return label_vec
  


# **Predicting test sentences**

In [ ]:
# Build the model. Don't use the model from sklearn

predicted_test_labels = naive_bayes_model(test_sentences.to_list(), pos_dict, neg_dict)


# **Calculating Accuracy**

In [ ]:
# Test the model on test set and report Accuracy


def get_accuracy(predicted_label, actual_label):
  """
  Arguments:
    predicted_label: list (List of the predicted labels corrosponding to the test_sentences)
    actual_label: list (List of the actual labels corrosponding to the test_sentences)
  
  Return: 
    returns accuracy
  """
  score = 0

  num_of_labels = len(actual_label)

  for i in range(num_of_labels):

    if predicted_label[i] == actual_label[i]:
      score += 1

  return (score/num_of_labels)


accuracy = get_accuracy(predicted_test_labels, test_labels)

print("The accuracy of the model is : ", accuracy)



The accuracy of the model is :  0.8442


# *LSTM* based Classifier

Use the above train and test splits.

In [ ]:
# Hyperparameters of the model
vocab_size = 3000 # choose based on statistics
oov_tok = '<OOK>'
embedding_dim = 100
max_length = 250 # choose based on statistics, for example 150 to 200
padding_type='post'
trunc_type='post'

In [ ]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences.to_list())
word_index = tokenizer.word_index

# convert train dataset to sequence and pad sequences
train_sequences = tokenizer.texts_to_sequences(train_sentences.to_list())
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

# convert Test dataset to sequence and pad sequences
test_sequences = tokenizer.texts_to_sequences(test_sentences.to_list())
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [ ]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 100)          300000    
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               84480     
_________________________________________________________________
dense (Dense)                (None, 24)                3096      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 25        
Total params: 387,601
Trainable params: 387,601
Non-trainable params: 0
_________________________________________________________________


In [ ]:
num_epochs = 5
history = model.fit(train_padded, train_labels, 
                    epochs=num_epochs, verbose=1, 
                    validation_split=0.1)

Epoch 1/5
1125/1125 [==============================] - 309s 271ms/step - loss: 0.3947 - accuracy: 0.8270 - val_loss: 0.3286 - val_accuracy: 0.8727
Epoch 2/5
1125/1125 [==============================] - 303s 269ms/step - loss: 0.2873 - accuracy: 0.8841 - val_loss: 0.2955 - val_accuracy: 0.8830
Epoch 3/5
1125/1125 [==============================] - 301s 268ms/step - loss: 0.2504 - accuracy: 0.8995 - val_loss: 0.2946 - val_accuracy: 0.8788
Epoch 4/5
1125/1125 [==============================] - 301s 268ms/step - loss: 0.2191 - accuracy: 0.9154 - val_loss: 0.3270 - val_accuracy: 0.8723
Epoch 5/5
1125/1125 [==============================] - 304s 270ms/step - loss: 0.1947 - accuracy: 0.9257 - val_loss: 0.3407 - val_accuracy: 0.8827


# **Predicting test sentences**

In [ ]:
# Get probabilities
prediction = model.predict(test_padded)

# Get labels based on probability 1 if p>= 0.5 else 0
predicted_labels = []

for p in prediction:
  if p >= 0.5:
    predicted_labels.append(1)
  
  else:
    predicted_labels.append(0)

# **Accuracy Calculation**

In [ ]:
# Calculate accuracy on Test data

# Accuracy : one can use classification_report from sklearn
target_names = ["negative", "positive"]
accuracy = classification_report(test_labels, predicted_labels, target_names = target_names)
print(accuracy)

              precision    recall  f1-score   support

    negative       0.88      0.87      0.88      5000
    positive       0.87      0.88      0.88      5000

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



# **Get predictions for random examples**

In [ ]:
# reviews on which we need to predict
sentence = ["The movie was very touching and heart whelming", 
            "I have never seen a terrible movie like this", 
            "the movie plot is terrible but it had good acting"]

# convert to a sequence

test_df = pd.DataFrame(sentence, columns = ['reviews'])

sentence = test_df['reviews']
sentence = convert_to_lower(sentence)
sentence = remove_non_alpha(sentence)
sentence = remove_extra_space(sentence)
sentence = remove_stopword(sentence)
sentence = lemmatize_and_stem_text(sentence)

sequences = tokenizer.texts_to_sequences(sentence)

# pad the sequence
padded = pad_sequences(sequences, padding='post', maxlen=max_length)

# Get probabilities
random_example_prediction = model.predict(padded)
print(random_example_prediction)

# Get labels based on probability 1 if p>= 0.5 else 0
random_example_predicted_labels = []

for p in random_example_prediction:

  if p >= 0.5:
    random_example_predicted_labels.append(1)
  
  else:
    random_example_predicted_labels.append(0)

print(random_example_predicted_labels)


[[0.80350554]
 [0.21996468]
 [0.07845634]]
[1, 0, 0]
